[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/open-atmos/PyPartMC-examples/blob/main/notebooks/condense.ipynb)   
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PyPartMC-examples/blob/main/notebooks/condense.ipynb)    
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PyPartMC-examples.git/main?urlpath=lab/tree/notebooks/condense.ipynb)

In [ ]:
# This file is a part of PyPartMC licensed under the GNU General Public License v3
# Copyright (C) 2023 University of Illinois Urbana-Champaign
# Authors:
#  - https://github.com/compdyn/partmc/graphs/contributors
#  - https://github.com/open-atmos/PyPartMC/graphs/contributors

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip --verbose install PyPartMC

In [ ]:
import PyPartMC as ppmc
from PyPartMC import si
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
gas_data = ppmc.GasData(
    ("H2SO4", "HNO3", "HCl", "NH3", "NO", "NO2")
)

In [ ]:
env_state = ppmc.EnvState({
    "rel_humidity": 0.95,
    "latitude": 0,
    "longitude": 0,
    "altitude": 0 * si.m,
    "start_time": 81000 * si.s,
    "start_day": 200
})

In [ ]:
aero_data = ppmc.AeroData((
    #         density  ions in soln (1) molecular weight    kappa (1)
    #         |                     |   |                   |
    {"SO4":  [1800 * si.kg/si.m**3, 1,  96.0 * si.g/si.mol, 0.00]},
    {"NO3":  [1800 * si.kg/si.m**3, 1,  62.0 * si.g/si.mol, 0.00]},
    {"Cl":   [2200 * si.kg/si.m**3, 1,  35.5 * si.g/si.mol, 0.00]},
    {"NH4":  [1800 * si.kg/si.m**3, 1,  18.0 * si.g/si.mol, 0.00]},
    {"MSA":  [1800 * si.kg/si.m**3, 0,  95.0 * si.g/si.mol, 0.53]},
    {"ARO1": [1400 * si.kg/si.m**3, 0, 150.0 * si.g/si.mol, 0.10]},
    {"ARO2": [1400 * si.kg/si.m**3, 0, 150.0 * si.g/si.mol, 0.10]},
    {"ALK1": [1400 * si.kg/si.m**3, 0, 140.0 * si.g/si.mol, 0.10]},
    {"OLE1": [1400 * si.kg/si.m**3, 0, 140.0 * si.g/si.mol, 0.10]},
    {"API1": [1400 * si.kg/si.m**3, 0, 184.0 * si.g/si.mol, 0.10]},
    {"API2": [1400 * si.kg/si.m**3, 0, 184.0 * si.g/si.mol, 0.10]},
    {"LIM1": [1400 * si.kg/si.m**3, 0, 200.0 * si.g/si.mol, 0.10]},
    {"LIM2": [1400 * si.kg/si.m**3, 0, 200.0 * si.g/si.mol, 0.10]},
    {"CO3":  [2600 * si.kg/si.m**3, 1,  60.0 * si.g/si.mol, 0.00]},
    {"Na":   [2200 * si.kg/si.m**3, 1,  23.0 * si.g/si.mol, 0.00]},
    {"Ca":   [2600 * si.kg/si.m**3, 1,  40.0 * si.g/si.mol, 0.00]},
    {"OIN":  [2600 * si.kg/si.m**3, 0,   1.0 * si.g/si.mol, 0.10]},
    {"OC":   [1400 * si.kg/si.m**3, 0,   1.0 * si.g/si.mol, 0.10]},
    {"BC":   [1800 * si.kg/si.m**3, 0,   1.0 * si.g/si.mol, 0.00]},
    {"H2O":  [1000 * si.kg/si.m**3, 0,  18.0 * si.g/si.mol, 0.00]}
))

In [ ]:
gas_state = ppmc.GasState(gas_data)
print(gas_state.mix_rats)

In [ ]:
times = [0 * si.s, 12*3600 * si.s]
rates = [1 / si.s, 1 / si.s]
null_gas = [{"time": times}, {"rate": rates}, {}]

AERO_MODE_CTOR_LOG_NORMAL = {
    "test_mode": {
        "mass_frac": [{"OIN": [.8]}, {"OC": [.1]},{"LIM1": [.1]}],
        "diam_type": "geometric",
        "mode_type": "log_normal",
        "num_conc": 1000000 / si.m**3,
        "geom_mean_diam": .2 * si.um,
        "log10_geom_std_dev": np.log10(1.6),
    }
}

scenario = ppmc.Scenario(gas_data, aero_data, {
    "temp_profile": [
        {"time": times},
        {"temp": [290 * si.K, 290 * si.K]}
    ],
    "pressure_profile": [
        {"time": times},
        {"pressure": [1000 * si.hPa, 1000 * si.hPa]}
    ],
    "height_profile": [
        {"time": times},
        {"height": [200 * si.m, 200 * si.m]}
    ],
    "gas_emissions": null_gas,
    "gas_background": null_gas,
    "aero_emissions": [
        {"time": [0 * si.s]},
        {"rate": [1 / si.s]},
        {"dist": [AERO_MODE_CTOR_LOG_NORMAL]},
    ],
    "aero_background": [
        {"time": [0 * si.s]},
        {"rate": [0e-8]},
        {"dist": [AERO_MODE_CTOR_LOG_NORMAL]},
    ],
    "loss_function": "none"
})

In [ ]:
run_part_opt = ppmc.RunPartOpt({
    "do_coagulation": True,
    "coag_kernel": "brown",
    "do_parallel": False,
    "do_nucleation": False,
    "do_camp_chem": False,
    "do_condensation": False,
    "t_max": 86400 * si.s,
    "del_t": 60 * si.s,
    "t_output": 0,
    "t_progress": 0,
    "allow_halving": True,
    "allow_doubling": True
})

In [ ]:
camp_core = ppmc.CampCore()
photolysis = ppmc.Photolysis()

In [ ]:
N_PART = 1000
aero_state = ppmc.AeroState(N_PART, aero_data)
t_initial = 0.0
n_steps = int((24 * 3600)/ 60) # Should grab the information from the env_state: t_max / del_t
num_conc = np.zeros(n_steps+1)
num_conc[0] = aero_state.total_num_conc
mass_conc = np.zeros(n_steps+1)
mass_conc[0] = aero_state.total_mass_conc
oc = np.zeros(n_steps+1)
bc = np.zeros(n_steps+1)
so4 = np.zeros(n_steps+1)
for i_time in range(1,n_steps+1):
    ppmc.run_part_timestep(
        scenario, env_state, aero_data, aero_state,
        gas_data, gas_state, run_part_opt, camp_core, photolysis, i_time, t_initial)
    num_conc[i_time] = aero_state.total_num_conc
    mass_conc[i_time] = aero_state.total_mass_conc

In [ ]:
print(num_conc)
plt.plot(mass_conc,label='mass conc')
plt.ylabel('mass concentration')
plt.twinx()
plt.plot(num_conc, label='num conc')
plt.ylabel('number concentration');

In [ ]:
num_concs = aero_state.num_concs
for i_part in range(aero_state.__len__()):
    particle = aero_state.particle(i_part)
#    print('SO4 %5.5e LIM1 %5.5e OIN %5.5e OC %5.5e BC %5.5e'
#          %(particle.volumes[0] * num_concs[i_part],
#            particle.volumes[11] * num_concs[i_part],
#            particle.volumes[16] * num_concs[i_part],
#            particle.volumes[17] * num_concs[i_part],
#            particle.volumes[18] * num_concs[i_part]))

In [ ]:
## Things to add
# bin_histogram_2d

In [ ]:
#aero_state_init.copy(aero_state)
N_PART = 10000
aero_state = ppmc.AeroState(N_PART, aero_data)
t_initial = 0.0
num_conc_block = np.zeros(25)
num_conc_block[0] = aero_state.total_num_conc
i_time = 1
bin_grid = ppmc.BinGrid(100,'log',1e-9,1e-6)
dry_diameters = aero_state.dry_diameters
num_concs = aero_state.num_concs
dist = ppmc.histogram_1d(bin_grid,dry_diameters,num_concs)
plt.plot(bin_grid.centers,dist,label='initial')
plt.xscale('log')
for i_timeblock in range(1,25):
    i_next = i_time + 60
    ppmc.run_part_timeblock(
     scenario, env_state, aero_data, aero_state,
     gas_data, gas_state, run_part_opt, camp_core, photolysis, i_time, i_next, t_initial)
    num_conc_block[i_timeblock] = aero_state.total_num_conc
    i_time = i_next + 1
    #    if (mod(i_time,6)== 0):
    dry_diameters = aero_state.dry_diameters
    num_concs_temp = aero_state.num_concs
    dist = ppmc.histogram_1d(bin_grid,dry_diameters,num_concs_temp)
    if (np.mod(i_timeblock - 1 , 6)== 0):
        plt.plot(bin_grid.centers,dist,label='hr = %i' %i_timeblock)
        plt.xscale('log')
plt.legend();

In [ ]:
#N_PART = 1000
aero_state = ppmc.AeroState(N_PART, aero_data)
#aero_state_init.copy(aero_state)
num_conc_run_part = np.zeros(2)
num_conc_run_part[0] = aero_state.total_num_conc
ppmc.run_part(
     scenario, env_state, aero_data, aero_state,
     gas_data, gas_state, run_part_opt, camp_core, photolysis)
num_conc_run_part[1] = aero_state.total_num_conc

In [ ]:
plt.figure(figsize=(16,10))
t = np.linspace(0,24*3600,24*60+1)
t_hr = np.linspace(0,24*3600,25)
plt.plot(t,num_conc,'.',label='step')
plt.plot(t_hr,num_conc_block,'x',label='block')
plt.plot([0,24*3600],num_conc_run_part,'r+',label='all')
plt.xlabel('time (s)')
plt.ylabel('number concentration (# m$^{-3}$)')
plt.ylim([0,7e9])
#plt.xlim([0,24*3600])
plt.legend()
plt.xticks(ticks=np.linspace(0,24*3600,7))
plt.grid();
#plt.savefig('tot_num_conc.pdf')

In [ ]:
print(num_conc_block[0],num_conc[0],num_conc_run_part[0])
print(num_conc_block[-1],num_conc[-1],num_conc_run_part[-1])